In [ ]:
import nltk

# Download the VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
import praw
import time
from kafka import KafkaProducer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the Reddit API client
reddit = praw.Reddit(
    client_id='<CLIENT ID>',
    client_secret='<CLIENT SECRET>',
    user_agent='<APP AGENT>'
)

# Specify the subreddit you want to monitor
subreddit_name = "depression"
subreddit = reddit.subreddit(subreddit_name)

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [ ]:
# Format submission post
def format_submission(submission):
    formatted_sub = {
        'id': submission.id,
        'title': submission.title,
        'text': submission.selftext,
        'link': 'https://www.reddit.com{}'.format(submission.permalink),
        'subreddit': submission.subreddit.display_name,
        'type': 'submission'
    }

    return formatted_sub

In [ ]:
# Define a function to perform sentiment analysis on a given text
def analyze_sentiment(text):
    # Use the VADER sentiment analyzer
    sentiment_scores = sia.polarity_scores(text)

    # Classify based on compound score (customize the threshold as needed)
    if sentiment_scores['compound'] >= 0.2:
        return 'Positive'
    elif sentiment_scores['compound'] <= -0.2:
        return 'Negative'
    else:
        return 'Neutral'

In [ ]:
# Define a function to fetch and process Reddit data
def fetch_reddit_data(last_processed_submission_id=None):
    try:
        # Retrieve submissions from the subreddit
        submissions = subreddit.new(limit=5, params={'after': f't3_{last_processed_submission_id}'})  # Example: Fetch the 5 newest submissions

        # Process and analyze the data as needed
        for submission in submissions:
            # print(submission.title + ": ")
            sub = format_submission(submission)
            print(sub)

            author = submission.author
            if author:
                print("Author:", author.name)

            #Perform sentiment analysis on the submission title
            sentiment = analyze_sentiment(sub['text'])
            print("Sentiment:", sentiment)

            # if sentiment == 'Negative':
            #       send_dm_to_author(author.name, "Your post has been classified as negative. If you need assistance, please reach out.")
            # print(submission.selftext)

    except Exception as e:
        print("An error occurred: {}".format(e))

    return last_processed_submission_id


In [ ]:
# Define the polling interval in seconds (e.g., 1 hour = 3600 seconds)
polling_interval = 3600

# Periodically fetch Reddit data
while True:
    fetch_reddit_data()
    #time.sleep(polling_interval)
    break

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



{'id': '18bdvcr', 'title': 'I am not mentally well', 'text': 'I feel like my sanity is slowly slipping away. Work, bills, life everything is chipping my mind down constantly. I have a medication and maybe I need more but I have no psychiatrist right now because mine moved. \n\n\nI feel like I just need to escape all people for a month or two I wish I could actually do this', 'link': 'https://www.reddit.com/r/depression/comments/18bdvcr/i_am_not_mentally_well/', 'subreddit': 'depression', 'type': 'submission'}
Author: danielj1632
Sentiment: Positive
{'id': '18bdsuu', 'title': '"Just your imagination."', 'text': 'I\'m a 31M and diagnosed with Major Depressive Disorder, Generalized Anxiety Disorder, Social Anxiety Disorder, Autism, and Dysthymia. I\'ve been hospitalized in a psych ward four times since 2016. I also have a moderate to severe stutter.\n\nMy current psychiatrist told me that my mental health issues are just my "imagination" and "all in my head" and things aren\'t that bad. S